In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import glob
from datetime import datetime, timedelta
import sys

# =======================================
# 1. CONFIGURACIÓN DEL MONITOR
# =======================================

# 🚨 ESTA RUTA DEBE COINCIDIR con la ruta donde el Consumer guarda los resultados.
AGGREGATED_OUTPUT_PATH = "output_realtime_analysis/aggregated_data"
# Tiempo de espera entre cada chequeo de archivos nuevos (en segundos)
REFRESH_INTERVAL = 5 # <--- CAMBIADO A 5 SEGUNDOS
# Ventana de tiempo para monitorear archivos recientes (en segundos)
RECENT_WINDOW_SECONDS = 60 

# =======================================
# 2. ESTRUCTURAS DE DATOS Y PLOTTER
# =======================================

plt.ion() # Habilitar modo interactivo para actualización dinámica
fig, ax = plt.subplots(figsize=(12, 6))

# Diccionario persistente en memoria para acumular las ventas totales por zona
sales_by_zone = {} 
# Conjunto para rastrear qué archivos CSV (part-000...) ya fueron procesados
processed_files = set() 

def update_realtime_plot():
    """Genera y refresca el gráfico de barras dinámico (Top 10 Ventas por Zona)."""
    
    if not sales_by_zone:
        ax.clear()
        ax.text(0.5, 0.5, "Esperando datos del Consumer...", 
                horizontalalignment='center', verticalalignment='center', transform=ax.transAxes)
        ax.set_title(f"Visualizador Activo - {datetime.now().strftime('%H:%M:%S')}")
        plt.draw()
        plt.pause(0.001)
        return
        
    # Análisis: Top 10 Zonas (Requerimiento 4: Top Elementos)
    sorted_sales = sorted(sales_by_zone.items(), key=lambda item: item[1], reverse=True)[:10]
    zones = [item[0] for item in sorted_sales]
    sales = [item[1] for item in sorted_sales]
    
    # 2. Actualizar el gráfico
    ax.clear() 
    
    y_pos = np.arange(len(zones))
    ax.barh(y_pos, sales, color='#00A388') 
    ax.set_yticks(y_pos)
    ax.set_yticklabels(zones, fontsize=10)
    
    ax.set_title(f'Ventas Acumuladas de Uber por Zona (Top 10) - {datetime.now().strftime("%H:%M:%S")}')
    ax.set_xlabel('Total Uber Sales Acumuladas ($)')
    ax.set_ylabel('Zona de Recogida')
    ax.grid(axis='x', linestyle='--', alpha=0.7)
    
    plt.draw()
    plt.pause(0.001)

def load_new_data():
    """Busca y carga nuevos archivos CSV generados por Spark, actualizando el análisis."""
    global processed_files
    
    all_files = glob.glob(f"{AGGREGATED_OUTPUT_PATH}/part-*.csv")
    new_files = [f for f in all_files if f not in processed_files]
    
    new_data_count = 0
    
    for file_path in new_files:
        try:
            # Usar Pandas para cargar el nuevo archivo de lote
            df_new = pd.read_csv(file_path)
            
            for index, row in df_new.iterrows():
                zone = row['pickup_zone']
                sales = row['total_sales_zone'] if not pd.isna(row['total_sales_zone']) else 0
                
                sales_by_zone[zone] = sales_by_zone.get(zone, 0) + sales
                new_data_count += 1
            
            processed_files.add(file_path)
            
        except Exception as e:
            # Esto maneja el caso donde Spark aún no ha terminado de escribir el archivo
            pass # Ignoramos el error para intentarlo en el próximo ciclo

    return new_data_count

def monitor_recent_files():
    """Busca archivos creados en el AGGREGATED_OUTPUT_PATH en los últimos 60 segundos."""
    
    time_limit = datetime.now() - timedelta(seconds=RECENT_WINDOW_SECONDS)
    recent_files = []
    
    # Buscar todos los archivos de Spark, incluyendo .csv y _SUCCESS
    all_items = glob.glob(f"{AGGREGATED_OUTPUT_PATH}/*")
    
    for item in all_items:
        # Obtener el tiempo de modificación del archivo
        mod_time = datetime.fromtimestamp(os.path.getmtime(item))
        
        if mod_time >= time_limit and os.path.isfile(item):
            # Formato de la hora de modificación
            time_str = mod_time.strftime("%H:%M:%S")
            recent_files.append(f"[{time_str}] {os.path.basename(item)}")
            
    return recent_files

# =======================================
# 3. BUCLE PRINCIPAL DE MONITORIZACIÓN
# =======================================

if __name__ == "__main__":
    
    if not os.path.isdir(AGGREGATED_OUTPUT_PATH):
        print(f"ERROR: No se encontró el directorio de salida: '{AGGREGATED_OUTPUT_PATH}'.")
        print("Asegúrate de ejecutar el Kafka_Processor.py primero.")
        sys.exit(1)

    print("\n--- VISUALIZADOR DE DATOS EN TIEMPO REAL INICIADO ---")
    print(f"Monitoreando la carpeta: {AGGREGATED_OUTPUT_PATH} cada {REFRESH_INTERVAL} segundos.")
    
    # Inicializa la ventana de espera
    update_realtime_plot() 

    try:
        while True:
            # 1. Cargar nuevos datos y actualizar análisis
            num_updates = load_new_data()
            
            if num_updates > 0:
                print(f"\n[{datetime.now().strftime('%H:%M:%S')}] --- NUEVO LOTE PROCESADO ---")
                print(f"-> Procesados {num_updates} nuevos registros.")
                update_realtime_plot()
            
            # 2. Monitoreo de actividad de archivos (Últimos 60s)
            recent_activity = monitor_recent_files()
            
            print(f"\n--- Actividad de Archivos (Últimos {RECENT_WINDOW_SECONDS}s) ---")
            if recent_activity:
                for f in recent_activity:
                    print(f" [NUEVO] {f}")
            else:
                print(" No se detectó actividad de escritura reciente.")
            print("-" * 50)
            
            time.sleep(REFRESH_INTERVAL)

    except KeyboardInterrupt:
        print("\nVisualizador detenido por el usuario.")
        plt.close(fig)
    except Exception as e:
        print(f"Error fatal del visualizador: {e}")
        plt.close(fig)

📊 Monitor Kafka — seguimiento de eventos y latencia

⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no existe el archivo CSV. Esperando datos...
⚠️ Aún no e